# M4-10

Capital imports (less than perfect elastic)

|Markets  |   X1  |    X2  |    E1  |    M2 | KM  |    W |   CONSH |  CONSF  |
|---------|-------|--------|--------|-------| --- |------|---------|---------|
|P1       |  150  |        |   -50  |       |     | -100 |         |         |    
|P2       |       |    50  |        |    50 |     | -100 |         |         |
|PL       | -100  |   -20  |        |       |     |      |    120  |         | 
|PK       |  -50  |   -30  |        |       | 20  |      |     80  |         |   
|PW       |       |        |        |       |     |  200 |   -200  |         |   
|PFX      |       |        |   100  |   -50 | -10 |      |         |  -50    | 
|PR       |       |        |   -50  |       | -10 |      |         |   50    |    

In [1]:
using MPSGE

In [ ]:
M4_10 = MPSGEModel()


initial_tax = 0

@parameters(M4_10, begin 
    TM2, initial_tax, (description = "Tariff for good 2",)
end)

@sectors(M4_10, begin
    X1,	(description = "Production index for good 1",)
    X2,	(description = "Production index good 2",)
    E1,	(description = "Export level of good 1",)
    E2,	(description = "Export level of good 2",)
    M1,	(description = "Import level of good 1",)
    M2,	(description = "Import level of good 2",)
    KM,	(description = "Captial imports",)
    W,	(description = "Welfare index ",)
end)
@commodities(M4_10, begin
    P1,	(description = "Price index for good 1",)
    P2,	(description = "Price index for good 1",)
    PFX,	(description = "Read exchange rate index",)
    PW,	(description = "Welfare price index",)
    PL,	(description = "Wage index",)
    PK,	(description = "Capital rental index",)
    PR,	(description = "Rent which generates the export demand function",)
end)
@consumers(M4_10, begin
    CONSH,	(description = "Income level for representative home agent",)
    CONSF,	(description = "Income level for representative foreign agent",)
end)


@production(M4_10, X1, [s=1,t=0],begin
    @output(P1, 170, t)
    @input(PL, 120,s)
    @input(PK, 50, s)
end)

@production(M4_10, X2, [s=1,t=0], begin
    @output(P2, 50, t)
    @input(PL, 20, s)
    @input(PK, 30, s)
end)

@production(M4_10, E1, [s=1,t=0], begin
    @output(PFX, 70, t)
    @input(P1, 70, s)
end)

@production(M4_10, E2, [s=0,t=0], begin
    @output(PFX, 50*.99, t)
    @input(P2, 50, s)
end)


@production(M4_10, M1, [s=0,t=0], begin
    @output(P1, 50, t)
    @input(PFX, 100*1.01, s)
end)

@production(M4_10, M2, [s=0,t=0], begin
    @output(P2, 50, t)
    @input(PFX, 50, s, taxes = [Tax(CONSH, TM2)]) 
end)

@production(M4_10, KM, [s=1,t=0], begin
    @output(PK, 20, t)
    @input(PFX, 10, s)
    @input(PR, 10, s)
end)

@production(M4_10, W, [s=1,t=0], begin
    @output(PW, 200, t)
    @input(P1, 100, s)
    @input(P2, 100, s)
end)

@demand(M4_10, CONSH, begin
    @final_demand(PW, 200)
    @endowment(PL, 140)
    @endowment(PK,  60)
end)

@demand(M4_10, CONSF, begin
    @final_demand(PFX, 10)
    @endowment(PR, 10)
end);

# Benchmark

In [7]:
fix(PW, 1)

set_start_value(E2, 0)
set_start_value(M1, 0)

solve!(M4_10, cumulative_iteration_limit = 0)
generate_report(M4_10)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_E0A7.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 16

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X1)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.093000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,X1,1.0,0.0
2,W,1.0,0.0
3,X2,1.0,0.0
4,E1,1.0,0.0
5,KM,1.0,0.0
6,PL,1.0,0.0
7,PR,1.0,0.0
8,PW,1.0,0.0
9,P1,1.0,0.0


# Counterfactual

In [8]:
set_value!(TM2, .05)

solve!(M4_10)
generate_report(M4_10)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_3909.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 16

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.6420e+00             0.0e+00 (income_balance[CONSH)
    1     1     0    14 7.1473e-01  1.0e+00    0.0e+00 (zero_profit[X1)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 7.1473e-01           I 0.0e+00 5.5e-01 (zero_profit[X1)
    1     1     3     3 1.1169e-03  1.0e+00 SO 0.0e+00 8.3e-04 (zero_profit[KM)
    2     1     4     4 4.2564e-08  1.0e+00 SO 0.0e+00 3.3e-08 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 

Row,var,value,margin
,GenericV…,Float64,Float64
1,X1,0.832108,-8.52651e-14
2,W,0.983301,0.0
3,X2,1.60623,-1.42109e-14
4,E1,0.581428,1.42109e-14
5,KM,1.11918,-1.60196e-8
6,PL,0.931174,-1.37561e-11
7,PR,1.22237,3.25002e-8
8,PW,1.0,-3.586e-8
9,P1,0.9759,1.42109e-14


# Counterfactual

In [9]:
fix(KM, 1)

solve!(M4_10)
generate_report(M4_10)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_B280.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 15

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.6420e+00             0.0e+00 (income_balance[CONSH)
    1     1     0    13 6.8018e-01  1.0e+00    0.0e+00 (zero_profit[X1)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 6.8018e-01           I 0.0e+00 5.5e-01 (zero_profit[X1)
    1     1     3     3 4.1555e-04  1.0e+00 SO 0.0e+00 3.0e-04 (market_clearanc)
    2     1     4     4 3.4240e-12  1.0e+00 SO 0.0e+00 2.6e-12 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . .

Row,var,value,margin
,GenericV…,Float64,Float64
1,X1,0.852628,-4.26326e-14
2,W,0.99668,0.0
3,X2,1.48897,-4.9738e-14
4,E1,0.611676,0.0
5,KM,1.0,-2.32608
6,PL,0.931174,-2.55795e-12
7,PR,0.9759,0.0
8,PW,1.0,-2.32608
9,P1,0.9759,-1.42109e-14
